## Looking at azimuth oscillations as a function of mount speed.

Craig Lage - Mar 7, 2023

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
import pickle as pkl
import pandas as pd
import json
from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

In [ ]:
def mountSpeed(az, el):
    # Calculate theoretical drive speed vs Az/El
    t0 = Time('2023-01-01T00:00:00') # doesn't really matter when
    altAz0 = SkyCoord(AltAz(alt=el*u.deg, az=az*u.deg, obstime=t0, location=AUXTEL_LOCATION))
    RaDec = altAz0.transform_to(ICRS)
    t1 = t0 + TimeDelta(1.0, format='sec')
    altAz1 = RaDec.transform_to(AltAz(obstime=t1, location=AUXTEL_LOCATION))
    #print(altAz1)
    newAz = altAz1.az.deg
    if az < 5.0 and newAz > 355.0:
        delta = newAz -360.0 - az
    else:
        delta = newAz - az

    azVel = abs(delta) * 3600.0
    elVel = abs(altAz1.alt.deg - el) * 3600.0
    #print(az, altAz1.az.deg, el, altAz1.alt.deg)
    return(azVel, elVel)
    

## Now we get the json data from RubinTV

In [ ]:
# This file needs to be manually downloaded from RubinTV
filename = '/home/craiglagegit/DATA/stiction_vs_speed_20230302/auxtel_2023-03-02.json'
df = pd.read_json(filename)
df = df.transpose()
limit = 0.40 # Limit in arcseconds
df_bad = df[df['Mount motion image degradation'] > limit]
df_good = df[df['Mount motion image degradation'] <= limit]
mount_degradation = df['Mount motion image degradation']
total = np.count_nonzero(~pd.isna(mount_degradation.values))
print(f" There are {len(df_bad)} values > {limit} out of {total} values")

In [ ]:
columns_to_keep = ['Altitude', 'Azimuth', 'Exposure id', 'Mount motion image degradation', 'TAI']
df_bad = df_bad[columns_to_keep]
df_bad['Type'] = 'Other'
df_good = df_good[columns_to_keep]

## Now we manually look at the plots to determine the failure type

In [ ]:
# 
for index in df_bad.index.to_list():
    type = input(f"Which type is {index}? (C = Cogging, S = Stiction, W = Wind, O = Other")
    if type == "" or type == 'O':
        df_bad.loc[index]['Type'] = 'Other'
    if type == 'C':
        df_bad.loc[index]['Type'] = 'Cogging'
    if type == 'S':
        df_bad.loc[index]['Type'] = 'Stiction'
    if type == 'W':
        df_bad.loc[index]['Type'] = 'Wind'


In [ ]:
# Pickle the dataframe so I don't have to categorize them again
file = open('/home/craiglagegit/DATA/stiction_vs_speed_20230302/df.pkl', 'wb')
pkl.dump(df_bad, file)
file.close()


In [ ]:
# Now unpickle them
file = open('/home/craiglagegit/DATA/stiction_vs_speed_20230302/df.pkl', 'rb')
df_bad = pkl.load(file)
file.close()

## Now we plot the mount fails against the azimuth mount speed

In [ ]:
alts = df_bad['Altitude'].values
azs = df_bad['Azimuth'].values
radAzs = np.radians(azs.astype(float))
Naz = 100
Nel = 50
azimuths = np.linspace(0, 360, Naz)
els = np.linspace(5, 85, Nel)
azValues = np.zeros([Naz, Nel])
elValues = np.zeros([Naz, Nel])
for i, az in enumerate(azimuths):
    for j, el in enumerate(els):
        [azSpeed, elSpeed] = mountSpeed(az, el)
        azValues[i,j] = azSpeed
        elValues[i,j] = elSpeed
        #print(az, el, mountSpeed(az, el)[0])
    
r, theta = np.meshgrid(els, np.radians(azimuths))



In [ ]:
#-- Plot... ------------------------------------------------
fig=plt.figure(figsize=(16,16))
#plt.subplots_adjust(wspace = 1.0)
vmin = -2.0
vmax = 2.0
levels = np.linspace(vmin, vmax, 9)
ax1 = plt.subplot(121, projection='polar')
ax1.set_title("Log10(Theoretical Azimuth speed (arcseconds/sec))")
ax1.invert_yaxis()
contourf_1 = ax1.contourf(theta, r, np.log10(azValues), levels=levels)
cax1 = fig.add_axes([ax1.get_position().x1+0.01,ax1.get_position().y0,0.02,ax1.get_position().height])
plt.colorbar(contourf_1, cax=cax1)
ax1.scatter([np.pi], [-AUXTEL_LOCATION.lat.deg], color='cyan', marker='x', s=200, label="SCP")
ax1.scatter(radAzs, alts, color='red', marker='x', label="Mount motion > 0.4")
ax1.legend()
plt.savefig('/home/craiglagegit/DATA/stiction_vs_speed_20230302/Mount_Fails_Positions_20230302.pdf')

In [ ]:
badAlts = df_bad['Altitude'].values
badAzs = df_bad['Azimuth'].values
badAzSpeeds = []
for i in range(len(badAzs)):
    [azSpeed, elSpeed] = mountSpeed(badAzs[i], badAlts[i])
    badAzSpeeds.append(azSpeed)
goodAlts = df_good['Altitude'].values
goodAzs = df_good['Azimuth'].values
goodAzSpeeds = []
for i in range(len(goodAzs)):
    [azSpeed, elSpeed] = mountSpeed(goodAzs[i], goodAlts[i])
    goodAzSpeeds.append(azSpeed)
        

In [ ]:
bins = np.linspace(0.0,50.0,100)
plt.title("Mount speed histogram - Theoretical")
plt.hist(badAzSpeeds, bins, alpha=0.5, color='red', label='Bad')
plt.hist(goodAzSpeeds, bins, alpha=0.5, color='green', label='Good')
plt.ylim(0,30)
plt.legend()
plt.xlabel("Azimuth drive speed (arcseconds/sec)")
plt.savefig('/home/craiglagegit/DATA/stiction_vs_speed_20230302/Mount_Fails_Speed_Histogram_20230302.pdf')

## Now we look at the actual speeds from the EFD

In [ ]:
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
expIds_bad = df_bad['Exposure id'].values.astype('int')
bad_motor1_speeds = []
bad_motor2_speeds = []
for expId in expIds_bad:
    exp = butler.get('raw', detector=0, exposure=expId)
    mData = exp.getMetadata()
    start = Time(mData['DATE-BEG'], format='isot', scale='tai')
    end = Time(mData['DATE-END'], format='isot', scale='tai')
    mount_speed = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'],
                                              start.utc, end.utc)
    bad_motor1_speeds.append(abs(mount_speed['azimuthMotor1Velocity'].values.mean()*3600.0))
    bad_motor2_speeds.append(abs(mount_speed['azimuthMotor2Velocity'].values.mean()*3600.0))

expIds_good = df_good['Exposure id'].values.astype('int')
good_motor1_speeds = []
good_motor2_speeds = []
for expId in expIds_good:
    exp = butler.get('raw', detector=0, exposure=expId)
    mData = exp.getMetadata()
    start = Time(mData['DATE-BEG'], format='isot', scale='tai')
    end = Time(mData['DATE-END'], format='isot', scale='tai')
    mount_speed = await client.select_packed_time_series("lsst.sal.ATMCS.measuredMotorVelocity", ['azimuthMotor1Velocity', 'azimuthMotor2Velocity'],
                                              start.utc, end.utc)
    good_motor1_speeds.append(abs(mount_speed['azimuthMotor1Velocity'].values.mean()*3600.0))
    good_motor2_speeds.append(abs(mount_speed['azimuthMotor2Velocity'].values.mean()*3600.0))


In [ ]:
# Both motors have about the same speed
plt.scatter(bad_motor1_speeds, bad_motor2_speeds)

In [ ]:
# General agreement between EFD measured speed and theoretical speed, but more scatter than I would expect.
plt.scatter(bad_motor1_speeds, badAzSpeeds)

In [ ]:
bins = np.linspace(0.0,50.0,100)
plt.title("Mount speed histogram - EFD")
plt.hist(bad_motor1_speeds, bins, alpha=0.5, color='red', label='Bad')
plt.hist(good_motor1_speeds, bins, alpha=0.5, color='green', label='Good')
plt.ylim(0,30)
plt.legend()
plt.xlabel("Azimuth drive speed (arcseconds/sec)")
plt.savefig('/home/craiglagegit/DATA/stiction_vs_speed_20230302/Mount_Fails_EFD_Speed_Histogram_20230302.pdf')